In [147]:
import scipy.stats as stats
import pandas as pd
import numpy as np

# Вариант 1

Независимые переменные: расход в городе, расход на шоссе, мощность
Зависимая: цена

In [148]:
data = pd.read_csv("cars93.csv")
Y = data['Price']
features = ['Horsepower', 'MPG.city', 'MPG.highway']
X = data[features].copy()
X['Free coeff'] = [1] * len(X)
features.append("Free coeff")

In [149]:
X

,Horsepower,MPG.city,MPG.highway,Free coeff
0,140,25,31,1
1,200,18,25,1
2,172,20,26,1
3,172,19,26,1
4,208,22,30,1
...,...,...,...,...
88,109,17,21,1
89,134,21,30,1
90,178,18,25,1
91,114,21,28,1


In [150]:
A = X.T @ X

In [151]:
A

,Horsepower,MPG.city,MPG.highway,Free coeff
Horsepower,2176206,280948,373151,13376
MPG.city,280948,49426,63101,2080
MPG.highway,373151,63101,81293,2705
Free coeff,13376,2080,2705,93


In [152]:
beta = np.linalg.inv(A) @ X.T @ Y
beta.index = features

In [153]:
beta

Horsepower     0.131314
MPG.city      -0.038600
MPG.highway   -0.178859
Free coeff     6.688679
dtype: float64

In [154]:
y_pred = X @ beta

In [155]:
mse = 0
for i in range(len(X)):
    mse += (y_pred.iloc[i] - Y.iloc[i])**2
    print(p - Y[i])
mse /= len(X)
print("MSE: ", mse)

7.069357582490872
-10.930642417509127
-6.130642417509129
-14.73064241750913
-7.030642417509128
7.269357582490873
2.1693575824908713
-0.7306424175091273
-3.3306424175091287
-11.73064241750913
-17.13064241750913
9.569357582490872
11.569357582490872
7.869357582490872
7.069357582490872
6.669357582490871
6.369357582490871
4.169357582490871
-15.030642417509128
4.569357582490873
7.169357582490871
-6.530642417509128
13.769357582490873
11.669357582490871
9.669357582490871
3.969357582490872
7.369357582490872
-2.8306424175091287
10.769357582490873
3.6693575824908713
15.569357582490872
12.869357582490872
11.669357582490871
7.069357582490872
8.969357582490872
3.0693575824908734
2.7693575824908727
2.0693575824908734
14.569357582490872
10.469357582490872
3.1693575824908713
10.869357582490872
5.469357582490872
14.969357582490872
12.969357582490872
12.969357582490872
9.069357582490872
-24.930642417509127
-5.030642417509128
-12.23064241750913
-11.330642417509125
-13.13064241750913
14.669357582490871
11.

# Проверим подозрения
- Чем больше мощность тем больше цена - действительно

- Цена изменяется в зависимости от расхода в городе

Действительно получившихся коэффициентов видно, что этот действительно заметно меньше (по модулю) остальных. Для уверенности воспользуемся T-тестом

- $H_0: \beta_1 = 0$
- $H_1: \beta_1 \neq 0$

$$T = \frac{\hat{\beta}_1}{SE(beta_1)} \sim T(n - k)$$


In [159]:
t_stat = beta.iloc[1] / np.sqrt(np.linalg.inv(A)[1, 1])
print("T-statistic: ", t_stat)
alpha = 0.05
df = len(X) - len(features)
p_value = 2 * (1 - stats.t.cdf(abs(t_stat), df))
print("p-value: ", p_value)
if stats.t.ppf(alpha / 2, df) <= t_stat <= stats.t.ppf(1 - alpha / 2, df):
    print("Accept H0")
else:
    print("Reject H0")

T-statistic:  -0.6458300957586504
p-value:  0.5200511100925178
Accept H0


Коэффициенты при расходе в городе и расходн на шоссе одновременно равны 0
- $H_0 : \beta_1 = \beta_2 = 0$
- $H_1 : \overline{H_0}$

Для этого воспользуемся F-тестом для проверки значимости линейной регрессии

$$F = \frac{SSE_F - SSE_R}{df_R - df_F} \cdot \frac{df_F}{SSE_F} \sim F(k - 2, n - k)$$

- $SSE_F = \sum_1^n (y_i - \hat{y}^F_i)^2$
- $SSE_R = \sum_1^n (y_i - \hat{y}^R_i)^2$

In [157]:
restricted = features[:1] + features[-1:]
X_r = X[restricted]
beta_r = beta[restricted]
y_pred_r = X_r @ beta_r

sse_f = np.sum((y_pred - Y)**2)
sse_r = np.sum((y_pred_r - Y)**2)

n = len(X)
df_f = n - len(features)
df_r = n - (len(features) - len(restricted))

f_stat = (sse_r - sse_f) / (df_r - df_f) * (df_f / sse_f)
print("F statistic: ", f_stat)
print("P-value: ", 1 - stats.f.cdf(f_stat, df_r - df_f, df_f))

if f_stat < stats.f.ppf(0.95, df_r - df_f, df_f):
    print("Accept H0")
else:
    print("Reject H0")


F statistic:  49.66474832533822
P-value:  3.219646771412954e-15
Reject H0
